<a id="top"></a>
# Style Transfer Tutorial

## OpenVINO version check:
You are currently using the latest development version of Intel® Distribution of OpenVINO™ Toolkit. Alternatively, you can open a version of this notebook for the Intel® Distribution of OpenVINO™ Toolkit LTS version by running the cell below and following the link it generates.

In [ ]:
from qarpo import displayMultiversionURL
import os
displayMultiversionURL(os.path.abspath(""), "tutorial_style_transfer_sample.ipynb", "openvino-dev-latest", ['openvino-lts'])

## Prerequisites

This tutorial requires the following:
- All files are present and in the following directory structure:
    - **tutorial_style_transfer_sample.ipynb** - This Jupyter* Notebook
    - **nst_vgg19/nst_vgg19-symbol.bin and nst_vgg19/nst_vgg19-symbol.xml** - The IR files for the inference model created using Model Optimizer
    - **tubingen.jpg** - Test image
    - **starry_night.jpg** - Test image
    - **doc_*.png** - Images used in the documentation
- Optional: URL to user's image or video to run inference on

<br><div class=note><i><b>Note: </b>It is assumed that the server this tutorial is being run on has Jupyter* Notebook, the Intel® Distribution of OpenVINO™ toolkit, and other required libraries already installed.  If you download or copy to a new server, this tutorial may not run.</i></div>  

## Introduction

The purpose of this tutorial is to examine a sample application that was created using the [Intel® Distribution of OpenViNO™ toolkit (Open Visual Inference & Neural Network Optimization)](https://software.intel.com/openvino-toolkit).  This tutorial will go step-by-step through the necessary steps to demonstrate style transfer on images and video .  Style transfer, where the style of one image is transferred to another as if recreated using the same style, is performed using a pre-trained network and running it using the Intel® Distribution of OpenVINO™ toolkit Inference Engine.  Inference will be executed using the same CPU(s) running this Jupyter* Notebook.

The pre-trained model to be used for object detection is the ["VGG19"](https://github.com/dmlc/web-data/tree/master/mxnet/neural-style) which has already been converted to the necessary Intermediate Representation (IR) files needed by the Inference Engine (Conversion is not covered here, please see the [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/en-us/openvino-toolkit) documentation, specifically the **Converting a Style Transfer Model from MXNet** instructions for more details).  The model used will re-style input images to a an abstracted "brush stroked" version.

### Key concepts

Before going into the samples in the tutorial steps, first we will go over some key concepts that will be covered in this tutorial.

#### Intel® distribution of OpenVINO™ toolkit overview and terminology

Let us begin with a brief overview of the Intel® Distribution of OpenVINO™ toolkit and what this tutorial will be covering.  The Intel® Distribution of OpenVINO™ toolkit enables the quick deployment of convolutional neural networks (CNN) for heterogeneous execution on Intel® hardware while maximizing performance. This is done using the Intel® Deep Learning Deployment Toolkit (Intel® DLDT) included within the Intel® Distribution of OpenVINO™ toolkit with its main components shown below.

![image alt text](./doc_openvino_overview_image.png)

The basic flow is:

1. Use a tool, such as Caffe*, to create and train a CNN inference model

2. Run the created model through Model Optimizer to produce an optimized Intermediate Representation (IR) stored in files (`.bin` and `.xml`) for use with the Inference Engine

3. The User Application then loads and runs models on devices using the Inference Engine and the IR files  

This tutorial will focus on the last step, the User Application and using the Inference Engine to run a model on a CPU.

##### Using the inference engine

Below is a more detailed view of the User Application and Inference Engine:

![image alt text](./doc_inference_engine_image.png)

The Inference Engine includes a plugin library for each supported device that has been optimized for the Intel® hardware device CPU, GPU, and VPU.  From here, we will use the terms "device" and “plugin” with the assumption that one infers the other (e.g. CPU device infers the CPU plugin and vice versa).  As part of loading the model, the User Application tells the Inference Engine which device to target which in turn loads the associated plugin library to later run on the associated device. The Inference Engine uses “blobs” for all data exchanges, basically arrays in memory arranged according the input and output data of the model.

###### Inference engine API integration flow

Using the inference engine API follows the basic steps outlined briefly below.  The API objects and functions will be seen later in the sample code.

1. Load the plugin

2. Read the model IR

3. Load the model into the plugin

6. Prepare the input

7. Run inference

8. Process the output

More details on the Inference Engine can be found in the [Inference Engine Development Guide](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)

### Input preprocessing

Often, the dimensions of the input data does not match the required dimensions of the input data for the inference model.  A common example is an input video frame.  Before the image may be input to the inference model, the input must be preprocessed to match the required dimensions for the inference model as well as channels (i.e. colors) and batch size (number of images present).  The basic step performed is to resize the frame from the source dimensions to match the required dimensions of the inference model’s input, reorganizing any dimensions as needed.

This tutorial and the many samples in the Intel® Distribution of OpenVINO™ toolkit use OpenCV to perform resizing of input data.  The basic steps performed using OpenCV are:

1.  Resize image dimensions form image to model's input W x H:<br>
`    frame = cv2.resize(image, (w, h))`
   
2. Change data layout from (H x W x C) to (C x H x W)<br>
`    frame = frame.transpose((2, 0, 1))`  

3. Reshape to match input dimensions<br>
`    frame = frame.reshape((n, c, h, w))`

## Sample application

The following sections will guide you through a sample application

### Imports

We begin by importing all of the Python* modules that will be used by the sample code:
- [os](https://docs.python.org/3/library/os.html#module-os) - Operating system specific module (used for file name parsing)
- [cv2](https://docs.opencv.org/trunk/) - OpenCV module
- [time](https://docs.python.org/3/library/time.html#module-time) - time tracking module (used for measuring execution time)
- [numpy](http://www.numpy.org/) - n-dimensional array manipulation
- [openvino.inference_engine](https://software.intel.com/en-us/articles/OpenVINO-InferEngine) - the IENetwork and IECore objects
- [matplotlib](https://matplotlib.org/) - pyplot is used for displaying output images

Run the cell below to import Python dependencies needed for displaying the results in this notebook. 

<br><div class=tip><b>Tip: </b>Select a cell and then use **Ctrl+Enter** to run that cell.</div>

In [ ]:
import os
import cv2
import time
import numpy as np
from openvino.inference_engine import IECore
import ngraph as ng
%matplotlib inline
from matplotlib import pyplot as plt
print('Imported Python modules successfully.')

### Configuration
Here we will create and set the following configuration parameters used by the sample:  
* **model_xml** - Path to the .xml IR file of the trained model to use for inference
* **model_bin** - Path to the .bin IR file of the trained model to use for inference (derived from *model_xml*)
* **input_path** - Path to input image
* **device** - Specify the target device to infer on,  CPU, GPU, or MYRIAD is acceptable, however the device must be present.  For this tutorial we use "CPU" which is known to be present.
* **mean_val_r** - Mean value of red channel for mean value addition in postprocessing 
* **mean_val_g** - Mean value of green channel for mean value addition in postprocessing
* **mean_val_b** - Mean value of blue channel for mean value addition in postprocessing

We will set all parameters here only once except for `input_path` which we will change later to point to different images and video.

In [ ]:
# model IR files
model_xml = "nst_vgg19/nst_vgg19-symbol.xml"
model_bin = "nst_vgg19/nst_vgg19-symbol.bin"

# input image file
input_path = "tubingen.jpg"

# CPU extension library to use
cpu_extension_path = os.path.expanduser("~")+"/inference_engine_samples/intel64/Release/lib/libcpu_extension.so"

# device to use
device = "CPU"

# RGB mean values to add to results
mean_val_r = 123.68
mean_val_g = 116.779
mean_val_b = 103.939

print("Configuration parameters settings:"
     "\n\tmodel_xml=", model_xml,
      "\n\tmodel_bin=", model_bin,
      "\n\tinput_path=", input_path,
      "\n\tdevice=", device, 
      "\n\tmean_val_r=", mean_val_r,
      "\n\tmean_val_g=", mean_val_g,
      "\n\tmean_val_b=", mean_val_b )

### Create inference engine instance

Next we create the Inference Engine instance to be used by our application.


In [ ]:
# create Inference Engine instance
ie = IECore()
print("An Inference Engine object has been created")

### Create network

Here we create an IENetwork object and load the model's IR files into it. After loading the model, we check to make sure that all the model's layers are supported by the plugin we will use. We also check to make sure that the model's input and output are as expected for later when we run inference.

In [ ]:
# load network from IR files
net = ie.read_network(model=model_xml, weights=model_bin)
print("Loaded model IR files [",model_bin,"] and [", model_xml, "]\n")

# check to make sure that the plugin has support for all layers in the loaded model
supported_layers = ie.query_network(net, "CPU") 
ng_function = ng.function_from_cnn(net)
not_supported_layers = \
[node.get_friendly_name() for node in ng_function.get_ordered_ops() if node.get_friendly_name() not in supported_layers]
if len(not_supported_layers) != 0:
    log.error("Following layers are not supported by the plugin for specified device {}:\n {}".
                      format(device, ', '.join(not_supported_layers)))
    log.error("Please try to specify cpu extensions library path in sample's command line parameters using"
                      "--cpu_extension command line argument")
    sys.exit(1)

### Load model

Here we load the model network into the plugin so that we may run inference.  `exec_net` will be used later to actually run inference.  After loading, we store the names of the input (`input_blob`) and output (`output_blob`) blobs to use when accessing the input and output blobs of the model.  Lastly, we store the model's input dimensions into the following variables:
- `n` = input batch size
- `c` = number of input channels (here 1 channel per color R,G, and B)
- `h` = input height
- `w` = input width

In [ ]:
# load the model into the Inference Engine for our device
exec_net = ie.load_network(network=net, num_requests=2, device_name=device)

# store name of input and output blobs
input_blob = next(iter(net.input_info))
output_blob = next(iter(net.outputs))

# read the input's dimensions: n=batch size, c=number of channels, h=height, w=width
n, c, h, w = net.input_info[input_blob].input_data.shape
print("Loaded model into Inference Engine for device:", device, 
      "\nModel input dimensions: n=",n,", c=",c,", h=",h,", w=",w)

### Prepare input

Here we read and then prepare the input image by resizing and re-arranging its dimensions according to the model's input dimensions. We define the functions `loadInputImage()` and `resizeInputImage()` for the operations so that we may reuse them again later in the tutorial.

In [ ]:
# define function to load an input image
def loadInputImage(input_path, verbose = True):
    # globals to store input width and height
    global input_w, input_h
    
    # use OpenCV to load the input image
    cap = cv2.VideoCapture(input_path) 
    
    # store input width and height
    input_w = cap.get(3)
    input_h = cap.get(4)
    if verbose: print("Loaded input image [",input_path,"], resolution=", input_w, "w x ",input_h,"h")

    # load the input image
    ret, image = cap.read()
    del cap
    return image

# define function for resizing input image
def resizeInputImage(image, verbose = True):
    # resize image dimensions form image to model's input w x h
    in_frame = cv2.resize(image, (w, h))
    # Change data layout from HWC to CHW
    in_frame = in_frame.transpose((2, 0, 1))  
    # reshape to input dimensions
    in_frame = in_frame.reshape((n, c, h, w))
    if verbose: print("Resized input image from {} to {}".format(image.shape[:-1], (h, w)))
    return in_frame

# load image
image = loadInputImage(input_path)

# resize the input image
in_frame = resizeInputImage(image)

# display input image
print("Input image:")
plt.axis("off")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

### Run inference

Now that we have the input image in the correct format for the model, we now run inference on the input image that was previously set to `./tubingen.jpg`:
<img src="tubingen.jpg" style="width:150px"/>

In [ ]:
# save start time
inf_start = time.time()

# run inference
res = exec_net.infer(inputs={input_blob: in_frame})   

# calculate time from start until now
inf_time = time.time() - inf_start
print("Inference complete, run time: {:.3f} ms".format(inf_time * 1000))

### Process and display results

Now we process the inference results by sorting all the possible classes by the probability assigned during inference, then selecting and displaying the top `report_top_n` items.  We define the function `processAndDisplayResults()` so that we may use it again later in the tutorial to process results.

In [ ]:
# create function to process inference results
def processResults(res):
    # get output
    result = res[output_blob][0]
    
    # Change layout from CxHxW to HxWxC 
    result = np.swapaxes(result, 0, 2)
    result = np.swapaxes(result, 0, 1)
    
    # add RGB mean values to 
    result = result[::] + (mean_val_r, mean_val_g, mean_val_b)
    
    # Clip RGB values to [0, 255] range
    result[result < 0] = 0
    result[result > 255] = 255

    # Matplotlib expects normalized image with pixel RGB values in range [0,1].
    result = result / 255
    return result
    
# create function to process and display inference results
def processAndDisplayResults(res, orig_input_image, orig_input_path, verbose = True):
    # display original input image
    plt.figure()
    plt.axis("off")
    im_to_show = cv2.cvtColor(orig_input_image, cv2.COLOR_BGR2RGB)
    plt.imshow(im_to_show)
    
    # get output
    result = processResults(res)
       
    # Show styled image
    if verbose: print("Results for input image: {}".format(orig_input_path))
    plt.figure()
    plt.axis("off")
    plt.imshow(result)

processAndDisplayResults(res, image, input_path)
print("Processed and displayed inference output results.")

## Exercise #1: Run a different image

Now that we have seen all the steps, let us run them again on a different image.  We also define `inferImage()` to combine the input processing, inference, and processing and displaying results so that we may use it again later in the tutorial.

In [ ]:
# define function to prepare input and run inference
def inferImage(image, verbose = True):
    # prepare input
    in_frame = resizeInputImage(image, verbose)

    # run inference
    res = exec_net.infer(inputs={input_blob: in_frame})   
    return res

# define function to prepare input, run inference, and process inference results
def inferAndDisplayImage(image, verbose = True):

    # run inference
    res = inferImage(image)

    # process inference results 
    processAndDisplayResults(res, image, input_path, verbose)

# set path to different input image
input_path="starry_night.jpg"

# load input image
image = loadInputImage(input_path)

# infer image and display results
inferAndDisplayImage(image)

## Exercise #2: (Optional) Run your own image

Here you may run any image you would like by setting the `input_path` variable which may be set to a local file or URL.  A sample URL is provided as an example.

In [ ]:
# input_path may be set to a local file or URL
input_path = "https://github.com/dmlc/web-data/raw/master/mxnet/neural-style/input/IMG_4343.jpg"

# load input image
image = loadInputImage(input_path)

# infer image and display results
inferAndDisplayImage(image, input_path)

## Exercise #3: Running inference on video
We have seen how to run individual images, now how do we do video?  To run inference on video is much the same as for a single image except that a loop is necessary to process all the frames in the video.  In the code below, we use the same method of loading a video as we had for an image, but now include the while-loop to keep reading images until `cap.isOpened()` returns false or `cap.read()` sets `ret` to false:
```python
while cap.isOpened():
    # read video frame
    ret, im = cap.read()
   
    # break if no more video frames
    if not ret:
        break  
    ...
```

In [ ]:
# close and then re-import matplotlib to be able to update output images for video
plt.close()
%matplotlib notebook
#%matplotlib inline
from matplotlib import pyplot as plt
# disable axis display
plt.axis("off")

# input_path may be set to local file or URL 
input_path = "/data/reference-sample-data/object-detection-python/cars_1900.mp4"

print("Loading video [",input_path,"]")
# use OpenCV to load the input image
cap = cv2.VideoCapture(input_path) 

# store input width and height
input_w = cap.get(3)
input_h = cap.get(4)
print("Loaded input video [",input_path,"], resolution=", input_w, "w x ",input_h,"h")

# track frame count and set maximum
frame_num = 0
max_frame_num = 6
skip_num_frames = 200
last_frame_num = cap.get(cv2.CAP_PROP_FRAME_COUNT)
if last_frame_num < 1: last_frame_num = "unknown"

while cap.isOpened():
    # read video frame
    ret, image = cap.read()
   
    # break if no more video frames
    if not ret:
        break  
    
    frame_num += 1

    # skip skip_num_frames of frames, then infer max_frame_num frames from there
    if frame_num > skip_num_frames: 
        # infer image
        res = inferImage(image, False)

        # show original image then force re-draw to show new image
        plt.figure(1)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.gcf().canvas.draw()

        # process results
        result = processResults(res)        

        # display results
        plt.figure(2)
        plt.imshow(result)
        plt.gcf().canvas.draw()

    # display current frame number
    print("Frame #:", frame_num, "/", last_frame_num, end="\r")
    
    # limit number of frames, video can be slow and gets slower the more frames that are processed
    if frame_num >= (max_frame_num + skip_num_frames): 
        print("\nStopping at frame #", frame_num)
        break

print("\nDone.")

## Exercise #4: (Optional) Run your own video
If you would like to see inference run on your own video, you may do so by first setting `input_path` to a local file or URL and then re-executing the cell above.  For example, you could use this video: https://github.com/intel-iot-devkit/sample-videos/raw/master/worker-zone-detection.mp4 by replacing the `input_path="..."` line above with the line:

```python
input_path="https://github.com/intel-iot-devkit/sample-videos/raw/master/worker-zone-detection.mp4"
```

You can control which frame to start from by setting `skip_num_frames` which will skip that many frames.
You can also control how many frames to show by setting `max_frame_num`.
<br><div class=note><i><b>Note: </b>There are more videos available to choose from at: https://github.com/intel-iot-devkit/sample-videos/</i></div>

## Cleanup

Now that we are done running the sample, we clean up by deleting objects before exiting.

In [ ]:
del exec_net
del net
del ie

print("Resource objects removed")

## Next steps

- [More Jupyter Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials/) - additional sample application Jupyter* Notebook tutorials
- [Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - sample applications
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge

## About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>
